In [1]:
import pandas as pd

In [2]:
from google.colab import files
files.upload()

Saving musicals_base_final.csv to musicals_base_final.csv


{'musicals_base_final.csv': b',Title,Description,Cast&Crew,Duration,Minimum price,Rating\n0,Beetlejuice,"Directed by Alex Timbers (Moulin Rouge!), Beetlejuice\xc2\xa0tells the story of Lydia Deetz, a strange and unusual teenager whose whole life changes when she meets a recently deceased couple and a demon with a thing for stripes. And under its uproarious surface (six feet under, to be exact), it\xe2\x80\x99s a remarkably touching show about family, love, and making the most of every Day-O!","Eddie Perfect, Alex Timbers",150,69,83\n1,Chicago,"Set in the legendary city during the roaring \xe2\x80\x9cjazz hot\xe2\x80\x9d 20s, Chicago tells the story of two rival vaudevillian murderesses locked up in Cook County Jail. Nightclub star Velma\xe2\x80\x99s serving time for killing her husband and sister after finding the two in bed together. Driven chorus girl Roxie\xe2\x80\x99s been tossed in the joint for bumping off the lover she\xe2\x80\x99s been cheating on her husband with. Not one to r

In [ ]:
df=pd.read_csv('musicals_base_final.csv', encoding='utf-8')
df

In [4]:
df2=df.drop(['Unnamed: 0'], axis=1)
df2

,Title,Description,Cast&Crew,Duration,Minimum price,Rating
0,Beetlejuice,"Directed by Alex Timbers (Moulin Rouge!), Beet...","Eddie Perfect, Alex Timbers",150,69,83
1,Chicago,Set in the legendary city during the roaring “...,"Charlotte d’Amboise, Bianca Marroquin, James M...",150,49,83
2,Company,The quintessential musical comedy about the se...,"Katrina Lenk, Patti LuPone, Matt Doyle, Etai B...",165,39,86
3,Hamilton,Hamilton is the story of the unlikely Founding...,"Miguel Cervantes, Jin Ha, Mandy Gonzalez, Euan...",175,199,97
4,Harry Potter and the Cursed Child,"19 years after Harry, Ron, and Hermione saved ...","Steve Haggard, David Abeles, Jenny Jules, Dian...",210,59,90
...,...,...,...,...,...,...
152,Bat Boy,Based on events as chronicled in The Weekly W...,"Deven May, Kathy Brier, Kerry Butler, Daria Ha...",140,0,0
153,We Are The Tigers,With a new school year comes a new beginning f...,"Lauren Zakrin, Cathy Ang, Jenny Rose Baker, Ca...",120,0,82
154,Pippin,"The ensemble cast tells the story of Pippin, a...","Josh Kaufman, Carly Hughes, Charlotte d’Ambois...",150,0,88
155,Sunday in the Park with George,Sunday in the Park with George follows painter...,"Jake Gyllenhaal, Annaleigh Ashford, Ruthie Ann...",150,0,89


In [ ]:
#библиотеки
!pip3 -qq install nltk --progress-bar off
!pip3 -qq install pymorphy2 --progress-bar off
!pip3 install -U scikit-learn --progress-bar off
import nltk
import pymorphy2
import re
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
mus_dict={}
for index in df2:
  titels_and_matches=df2.iloc[index]['Title'].split()
  mus_dict[index]=titels_and_matches

TypeError: ignored

In [ ]:
nltk.download('punkt')

df['Title_tkn'] = df.apply(lambda row: nltk.word_tokenize(row['Title']), axis=1)
df['Description_tkn']=df.apply(lambda row: nltk.word_tokenize(row['Description']),axis=1)
df['Cast&Crew_tkn']=df.apply(lambda row: nltk.word_tokenize(row['Cast&Crew']),axis=1)

df

In [ ]:
#попытки токенизировать -- здесь и далее

from tokenizers import CharBPETokenizer
vocab = "vocab.json"
merges = "merges.txt"
tokenizer = CharBPETokenizer(vocab, merges)

encoded = tokenizer.encode("I can feel the magic, can you?")
print(encoded.ids)
print(encoded.tokens)

In [ ]:
tokenizer = CharBPETokenizer()
tokenizer.train(['musicals_base_final.csv'])

tokenizer.save("my-bpe.tokenizer.json")

In [ ]:
titles=df.apply(lambda row: CharBPETokenizer(row['Title']),axis=1)

titles

0      Tokenizer(vocabulary_size=0, model=BPE, unk_to...
1      Tokenizer(vocabulary_size=0, model=BPE, unk_to...
2      Tokenizer(vocabulary_size=0, model=BPE, unk_to...
3      Tokenizer(vocabulary_size=0, model=BPE, unk_to...
4      Tokenizer(vocabulary_size=0, model=BPE, unk_to...
                             ...                        
152    Tokenizer(vocabulary_size=0, model=BPE, unk_to...
153    Tokenizer(vocabulary_size=0, model=BPE, unk_to...
154    Tokenizer(vocabulary_size=0, model=BPE, unk_to...
155    Tokenizer(vocabulary_size=0, model=BPE, unk_to...
156    Tokenizer(vocabulary_size=0, model=BPE, unk_to...
Length: 157, dtype: object

In [ ]:
titles=df.apply(lambda row: nltk.word_tokenize(row['Title']),axis=1)
titles
cc=df.apply(lambda row: nltk.word_tokenize(row['Cast&Crew']),axis=1)
cc

0                     [Eddie, Perfect, ,, Alex, Timbers]
1      [Charlotte, d, ’, Amboise, ,, Bianca, Marroqui...
2      [Katrina, Lenk, ,, Patti, LuPone, ,, Matt, Doy...
3      [Miguel, Cervantes, ,, Jin, Ha, ,, Mandy, Gonz...
4      [Steve, Haggard, ,, David, Abeles, ,, Jenny, J...
                             ...                        
152    [Deven, May, ,, Kathy, Brier, ,, Kerry, Butler...
153    [Lauren, Zakrin, ,, Cathy, Ang, ,, Jenny, Rose...
154    [Josh, Kaufman, ,, Carly, Hughes, ,, Charlotte...
155    [Jake, Gyllenhaal, ,, Annaleigh, Ashford, ,, R...
156    [Ava, Briglia, ,, Bryce, Ryness, ,, Jennifer, ...
Length: 157, dtype: object

In [ ]:
# df[:].apply(CharBPETokenizer)
df['Description'].apply(CharBPETokenizer)
# df['Cast&Crew'].apply(CharBPETokenizer)
# df['Duration'].apply(CharBPETokenizer)
# df['Minimum price'].apply(CharBPETokenizer)
# df['Rating'].apply(CharBPETokenizer)

In [ ]:
def preprocess_tokenize(data):
data_preprocessed_tokenized=[]



In [ ]:
#токенизация
df=pd.DataFrame(['Title']['Description']['Cast&Crew']['Duration'],['Minimum price'],['Rating'])
print(df)
# def preprocess_tokenize
#   df_x=[]
#   for x in (df[:][0]):
#     df_token=CharBPETokenizer(x)
#   return df_x

# print(preprocess_tokenize(df))
# # df['tokenized_text']=df['Title','Description','Cast&Crew','Duration','Minimum price','Rating'].apply(CharBPETokenizer)
#   df['Description'].apply(CharBPETokenizer)
#   df['Cast&Crew'].apply(CharBPETokenizer)
#   df['Duration'].apply(CharBPETokenizer)
#   df['Minimum price'].apply(CharBPETokenizer)
#   df['Rating'].apply(CharBPETokenizer)
# #   # return df_preprocessed_tokenized

In [ ]:
#попытка создания словаря

df=pd.DataFrame(data['part'][0]['comment'])
split_data = df["comment"].str.split(" ")
data = split_data

print(data)

def tokenization_s(data): # same can be achieved for words tokens
    s_new = []
    for sent in (data[:][0]): #For NumpY = sentences[:]
        s_token = sent_tokenize(sent)
        if s_token != '':
            s_new.append(s_token)
    return s_new

print(tokenization_s(data))

In [ ]:
heads=(titles)